In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_diabetes_df, load_breast_cancer_df
from utils.preprocessing import preprocess_df
from sklearn.model_selection import train_test_split
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.save import save_result_as_csv
from utils.gs import generate_gs_result, process_results


pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

seed = 123
# tf.random.set_seed(seed)
# np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
#### Select dataset ####

dataset_name = 'breast_cancer' 

if  dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
### Load models.
models = load_three_models(X_train.shape[-1], dataset_name)

In [7]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.9737] | RF [0.9912] | NN [0.9737]


In [8]:
### Setting up the CF generating amount.
num_instances = 5
num_cf_per_instance = 1

In [9]:
results = generate_gs_result(df_info, test_df, models, num_instances, num_cf_per_instance, 2000)
result_dfs = process_results(df_info, results)

Finding counterfactual for dt
instance 0
CF 0
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  12
Final radius:  (0.20799999999999996, 0.22599999999999995)
Final number of ennemies:  1
Feature selection...
Reduced 29 coordinates
instance 1
CF 0
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  15
Final radius:  (0.26199999999999996, 0.27999999999999997)
Final number of ennemies:  3
Feature selection...
Reduced 29 coordinates
instance 2
CF 0
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  7
Final radius:  (0.11800000000000001, 0.136)
Final number of ennemies:  2
Feature selection...
Reduced 28 coordinates
instance 3
CF 0
647 ennemies found in initial sphere. Zooming in...
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  6
Final radius:  (0.01, 0.0118)
Final number of ennemies:  3
Feature selection...
Reduced 29 coordi

In [10]:
### Save result as file.
save_result_as_csv("GS_lp", dataset_name, result_dfs)

Result has been saved to ./results/GS_breast_cancer
